In [1]:
import psycopg2
from configparser import ConfigParser

In [53]:
config = ConfigParser()
config.read('config.ini')

# Установка пути к нужной схеме 
def search_path(conn):
    with conn.cursor() as cursor:
        cursor.execute(
        """SET search_path TO sql_py;""")
        conn.commit()

# Создание базы (1)
# Телефоны хранятся в отдельной таблице
def create_db(conn):
    with conn.cursor() as cursor:
        search_path(conn)
        cursor.execute("""
            create table if not exists client(
            client_id serial primary key,
            first_name varchar(50) not null,
            last_name varchar(50) not null,
            email varchar(50) not null unique
        );
        """) 
        cursor.execute("""
            create table if not exists phone(
            phone_id serial primary key,
            client_id integer not null references client(client_id),
            phone_number varchar(20) not null unique
        );
        """) 
        conn.commit()
        print('Database created')


# Поиск id существующего клиента (7)
def find_client(conn, first_name=None, last_name=None, email=None, phone=None):
    with conn.cursor() as cursor:
        search_path(conn)
        conn.commit()
        if phone is None:
            cursor.execute("""
                select client_id
                from client 
                where first_name = %s and 
                last_name = %s and email = %s;""", 
            (first_name, last_name, email)) 
            conn.commit()
            return cursor.fetchone()[0]
        else:    
            cursor.execute("""
                select client_id
                from phone 
                where phone_number = %s;""", 
            (phone,)) 
            conn.commit()
            return cursor.fetchone()[0]


# Добавление клиента (2)
# Сначала заполняется таблица клиентов, затем посредством предудыщей функции определяется 
# id вновь добавленного клиента, которому в таблице телефонов присваиваются все его номера
def add_client(conn, first_name, last_name, email, phones=None):
    with conn.cursor() as cursor:
        search_path(conn)
        cursor.execute("""
            insert into client (first_name, last_name, email)
            values (%s, %s, %s);""", 
            (first_name, last_name, email))
        conn.commit()
        
        if phones is not None:
            id = find_client(conn, first_name, last_name, email)
            for phone in phones:
                cursor.execute("""
                    insert into phone (client_id, phone_number)
                    values (%s, %s);""",
                    (id, phone)) 
        conn.commit()
        print('Client added to database')


# Добавление телефона для существующего клиента (3)
def add_phone(conn, client_id, phone):
    with conn.cursor() as cursor:
        search_path(conn)
        cursor.execute("""
            insert into phone (client_id, phone_number)
            values (%s, %s);""",
            (client_id, phone)) 
        conn.commit()
        print('Phone added to database')


# Удаление телефона для существующего клиента (5)
def delete_phone(conn, client_id, phone):
    with conn.cursor() as cursor:
        search_path(conn)
        cursor.execute("""
            delete from phone 
            where client_id = %s and phone_number = %s;""",
            (client_id, phone)) 
        conn.commit()
        print('Phone deleted from database')


# Удаление существующего клиента (6)
def delete_client(conn, client_id):
    with conn.cursor() as cursor:
        search_path(conn)
        cursor.execute("""
            delete from phone 
            where client_id = %s;""",
            (client_id,))
        cursor.execute("""
            delete from client 
            where client_id = %s;""",
            (client_id,))
        conn.commit()
        print('Client deleted from database')


# Изменение данных о клиенте (4)
def change_client(conn, client_id, first_name=None, last_name=None, email=None, phones=None):
    with conn.cursor() as cursor:
        search_path(conn)
        if first_name is not None:
            cursor.execute("""
                update client set first_name = %s
                where client_id = %s;""",
                (first_name, client_id)) 
        if last_name is not None:
            cursor.execute("""
                update client set last_name = %s
                where client_id = %s;""",
                (last_name, client_id))
        if email is not None:
            cursor.execute("""
                update client set email = %s
                where client_id = %s;""",
                (email, client_id))
        if phones is not None:
            cursor.execute("""
                delete from phone
                where client_id = %s;""",
                (client_id, ))
            for phone in phones:
                cursor.execute("""
                    insert into phone (client_id, phone_number)
                    values (%s, %s);""",
                    (client_id, phone))
        conn.commit()
        print('All changes are made')

In [ ]:
with psycopg2.connect(
    host=config['db_task']['host'],
    database=config['db_task']['database'],
    user=config['db_task']['user'],
    password=config['db_task']['password'],
) as conn:
    #create_db(conn)
    #add_client(conn, 'first_name_3', 'last_name_3', 'email_3', ['+71444555', '+715556666'])
    #print(find_client(conn, phone='+7444555'))
    #delete_phone(conn, 5, '+715556666')
    #change_client(conn, 1, first_name='first_name_0', phones=['+71245678', '+71987654'])

conn.close()